In [1]:
#Classifying Fashion-MNIST dataset
#Import the libraries
import torch
from torch import nn
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Import the dataset
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,),(0.5,))])

trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [3]:
image, label = next(iter(trainloader))
image.shape

torch.Size([64, 1, 28, 28])

In [4]:
#Building the network
import torch.nn.functional as F
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [5]:
#Training the model
from torch import optim

model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 5
for e in range(epochs):
    print("Pass ", e+1, " ...")
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    else:
        print("Loss ", running_loss/len(trainloader))

Pass  1  ...
Loss  0.5192710626354096
Pass  2  ...
Loss  0.39610210072193575
Pass  3  ...
Loss  0.35433156731159193
Pass  4  ...
Loss  0.3350132057534606
Pass  5  ...
Loss  0.3174164085102869


In [32]:
#Calculating the test result
dataiter = iter(testloader)
images, labels = dataiter.next()
logps = model(images)
ps = torch.exp(logps)
results = [y.argmax() for y in ps]
print(results)
print(labels)

[tensor(0), tensor(3), tensor(2), tensor(9), tensor(9), tensor(2), tensor(9), tensor(9), tensor(9), tensor(1), tensor(2), tensor(8), tensor(3), tensor(6), tensor(7), tensor(8), tensor(8), tensor(5), tensor(4), tensor(1), tensor(0), tensor(7), tensor(9), tensor(9), tensor(2), tensor(3), tensor(5), tensor(5), tensor(3), tensor(3), tensor(5), tensor(1), tensor(6), tensor(0), tensor(0), tensor(2), tensor(2), tensor(3), tensor(8), tensor(0), tensor(2), tensor(0), tensor(6), tensor(6), tensor(8), tensor(7), tensor(8), tensor(7), tensor(9), tensor(1), tensor(8), tensor(8), tensor(9), tensor(6), tensor(3), tensor(6), tensor(7), tensor(3), tensor(4), tensor(7), tensor(2), tensor(8), tensor(6), tensor(1)]
tensor([0, 3, 2, 9, 9, 4, 9, 9, 9, 1, 2, 8, 3, 2, 7, 8, 8, 5, 4, 1, 0, 7, 9, 9,
        2, 3, 5, 5, 1, 3, 7, 1, 6, 0, 0, 2, 2, 3, 8, 0, 2, 0, 2, 6, 8, 7, 8, 7,
        9, 1, 8, 8, 9, 6, 3, 6, 7, 3, 4, 7, 2, 8, 6, 1])
